In [113]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable

train = pd.read_csv('input/train.csv')
test  = pd.read_csv('input/test.csv')
df_sub   = pd.read_csv('input/gender_submission.csv')

In [27]:
# Создадим два новых dataframes
df_bin = pd.DataFrame() 
df_con = pd.DataFrame()

In [30]:
df_bin['Survived'] = train['Survived']
df_con['Survived'] = train['Survived']

In [31]:
df_bin['Pclass'] = train['Pclass']
df_con['Pclass'] = train['Pclass']

In [32]:
df_bin['Sex'] = train['Sex']
df_bin['Sex'] = np.where(df_bin['Sex'] == 'female', 1, 0)
df_con['Sex'] = train['Sex']

In [33]:
df_bin['SibSp'] = train['SibSp']
df_con['SibSp'] = train['SibSp']

In [34]:
df_bin['Parch'] = train['Parch']
df_con['Parch'] = train['Parch']

In [35]:
df_con['Name'] = train['Name']
df_con.head(10)
train_test_data = [df_con]
for title in train_test_data:
    title['Title'] = title['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [37]:
df_bin['Age'] = train['Age']
df_con['Age'] = train['Age']

In [39]:
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                 "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,"Countess": 3,
                 "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3, "Dona" : 3, "Mme": 3,"Capt": 3,"Sir": 3}
for bin_title in [df_con]:
    df_bin['Title'] = bin_title['Title'].map(title_mapping)

In [40]:
df_bin['Age'].fillna(df_bin.groupby('Title')['Age'].transform('median'), inplace=True)
df_con['Age'].fillna(df_con.groupby('Title')['Age'].transform('median'), inplace=True)

In [41]:
for bin_age in [df_bin]:
    bin_age.loc[bin_age['Age'] <= 15, 'Age'] = 0, # Дети
    bin_age.loc[(bin_age['Age'] > 15) & (bin_age['Age'] <= 25), 'Age'] = 1, # Молодые
    bin_age.loc[(bin_age['Age'] > 25) & (bin_age['Age'] <= 35), 'Age'] = 2, # Взрослые
    bin_age.loc[(bin_age['Age'] > 35) & (bin_age['Age'] <= 48), 'Age'] = 3, # Средний возраст
    bin_age.loc[(bin_age['Age'] > 48), 'Age'] = 4 # Пожилые

In [42]:
df_con['Embarked'] = train['Embarked']
for embarked_con in [df_con]:
    embarked_con['Embarked'] = embarked_con['Embarked'].fillna('S')

In [43]:
embarked_map = {'S': 0, 'C': 1, 'Q': 2}
df_bin['Embarked'] = df_con['Embarked']
for embarked_bin in [df_bin]:
    embarked_bin['Embarked'] = embarked_bin['Embarked'].map(embarked_map)

In [44]:
df_bin.Age = df_bin.Age.astype(np.int64)

In [45]:
df_bin['Fare'] = train['Fare']

In [46]:
for dataset in [df_bin]:
    dataset.loc[ dataset['Fare'] <= 17, 'Fare'] = 0,
    dataset.loc[(dataset['Fare'] > 17) & (dataset['Fare'] <= 30), 'Fare'] = 1,
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2,
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3

In [47]:
df_bin.Fare = df_bin.Fare.astype(np.int64)

In [54]:
test_  = pd.read_csv('input/test.csv')

In [171]:
test[wanted_test_columns].head()

,Pclass,Sex,SibSp,Parch,Age,Embarked,Fare
0,3,0,0,0,2,2,0
1,3,1,1,0,3,0,0
2,2,0,0,0,4,2,0
3,3,0,0,0,2,0,0
4,3,1,1,1,1,0,1


In [173]:
X_train = df_bin
y_train = train['Survived']

In [174]:
X_train = X_train.values
y_train = y_train.values

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [176]:
XTrain, yTrain, XTest, yTest = map(torch.tensor, (X_train, y_train, X_test, y_test))

In [177]:
print(XTrain.shape, yTrain.shape)

torch.Size([712, 7]) torch.Size([712])


In [178]:
XTrain = XTrain.float()
XTest = XTest.float()

In [188]:
print(XTrain.shape, yTrain.shape, yTest.shape, XTest.shape)

torch.Size([712, 7]) torch.Size([712]) torch.Size([179]) torch.Size([179, 7])


In [126]:
wanted_test_columns = df_bin.columns
wanted_test_columns

Index(['Pclass', 'Sex', 'SibSp', 'Parch', 'Age', 'Embarked', 'Fare'], dtype='object')

In [97]:
for embarked_bin in [test]:
    embarked_bin['Embarked'] = embarked_bin['Embarked'].map(embarked_map)

In [98]:
for bin_age in [test]:
    bin_age.loc[bin_age['Age'] <= 15, 'Age'] = 0, # Дети
    bin_age.loc[(bin_age['Age'] > 15) & (bin_age['Age'] <= 25), 'Age'] = 1, # Молодые
    bin_age.loc[(bin_age['Age'] > 25) & (bin_age['Age'] <= 35), 'Age'] = 2, # Взрослые
    bin_age.loc[(bin_age['Age'] > 35) & (bin_age['Age'] <= 48), 'Age'] = 3, # Средний возраст
    bin_age.loc[(bin_age['Age'] > 48), 'Age'] = 4 # Пожилые

In [99]:
for title in [test]:
    title['Title'] = title['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
for bin_title in [test]:
    bin_title['Title'] = bin_title['Title'].map(title_mapping)

In [100]:
test['Age'].fillna(test.groupby('Title')['Age'].transform('median'), inplace=True)

In [101]:
test['Fare'].fillna(test.groupby('Pclass')['Fare'].transform('median'), inplace=True)
for dataset in [test]:
    dataset.loc[ dataset['Fare'] <= 10, 'Fare'] = 0,
    dataset.loc[(dataset['Fare'] > 10) & (dataset['Fare'] <= 30), 'Fare'] = 1,
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2,
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3
test.Fare = test.Fare.astype(np.int64)

In [102]:
test['Sex'] = np.where(test['Sex'] == 'female', 1, 0)

In [104]:
test.Age = test.Age.astype(np.int64)

In [105]:
pred = torch.from_numpy(test[wanted_test_columns].values)

In [106]:
sample_sub = pd.read_csv('input/gender_submission.csv')

In [109]:
sample_sub['Survived'] = the_model(pred.float()).argmax(dim=1)

In [190]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=7, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [191]:
criterion = nn.CrossEntropyLoss()

In [187]:
len(X_train) // batch_size

11

In [192]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [195]:
batch_size = 64
n_epochs = 2000
batch_no = len(X_train) // batch_size

train_loss = 0
train_loss_min = np.Inf
for epoch in range(n_epochs):
    for i in range(batch_no):
        start = i*batch_size
        end = start+batch_size
        x_var = Variable(torch.FloatTensor(X_train[start:end]))
        y_var = Variable(torch.LongTensor(y_train[start:end])) 
        
        optimizer.zero_grad()
        output = model(x_var)
        loss = criterion(output,y_var)
        loss.backward()
        optimizer.step()
        
        values, labels = torch.max(output, 1)
        num_right = np.sum(labels.data.numpy() == y_train[start:end])
        train_loss += loss.item()*batch_size
    
    train_loss = train_loss / len(X_train)
    if train_loss <= train_loss_min:
        print("Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(train_loss_min,train_loss))
        torch.save(model.state_dict(), "model.pt")
        train_loss_min = train_loss
    
    if epoch % 200 == 0:
        print('')
        print("Epoch: {} \tTrain Loss: {} \tTrain Accuracy: {}".format(epoch+1, train_loss,num_right / len(y_train[start:end]) ))
print('Training Ended! ')

Validation loss decreased (   inf ===> 0.329211). Saving the model...

Epoch: 1 	Train Loss: 0.32921074749378676 	Train Accuracy: 0.78125
Validation loss decreased (0.329211 ===> 0.328321). Saving the model...
Validation loss decreased (0.328321 ===> 0.326182). Saving the model...
Validation loss decreased (0.326182 ===> 0.325281). Saving the model...
Validation loss decreased (0.325281 ===> 0.321435). Saving the model...
Validation loss decreased (0.321435 ===> 0.320603). Saving the model...
Validation loss decreased (0.320603 ===> 0.319098). Saving the model...
Validation loss decreased (0.319098 ===> 0.318781). Saving the model...
Validation loss decreased (0.318781 ===> 0.317622). Saving the model...

Epoch: 201 	Train Loss: 0.33328051551435217 	Train Accuracy: 0.78125
Validation loss decreased (0.317622 ===> 0.311174). Saving the model...
Validation loss decreased (0.311174 ===> 0.309825). Saving the model...

Epoch: 401 	Train Loss: 0.3523615774620969 	Train Accuracy: 0.796875
Va

In [197]:
model

Net(
  (fc1): Linear(in_features=7, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [132]:
for embarked_bin in [test]:
    embarked_bin['Embarked'] = embarked_bin['Embarked'].map(embarked_map)

In [133]:
for title in [test]:
    title['Title'] = title['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
for bin_title in [test]:
    bin_title['Title'] = bin_title['Title'].map(title_mapping)

In [134]:
test['Age'].fillna(test.groupby('Title')['Age'].transform('median'), inplace=True)

In [135]:
test['Fare'].fillna(test.groupby('Pclass')['Fare'].transform('median'), inplace=True)
for dataset in [test]:
    dataset.loc[ dataset['Fare'] <= 10, 'Fare'] = 0,
    dataset.loc[(dataset['Fare'] > 10) & (dataset['Fare'] <= 30), 'Fare'] = 1,
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2,
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3
test.Fare = test.Fare.astype(np.int64)

In [142]:
for bin_age in [test]:
    bin_age.loc[bin_age['Age'] <= 15, 'Age'] = 0, # Дети
    bin_age.loc[(bin_age['Age'] > 15) & (bin_age['Age'] <= 25), 'Age'] = 1, # Молодые
    bin_age.loc[(bin_age['Age'] > 25) & (bin_age['Age'] <= 35), 'Age'] = 2, # Взрослые
    bin_age.loc[(bin_age['Age'] > 35) & (bin_age['Age'] <= 48), 'Age'] = 3, # Средний возраст
    bin_age.loc[(bin_age['Age'] > 48), 'Age'] = 4 # Пожилые

In [136]:
test['Sex'] = np.where(test['Sex'] == 'female', 1, 0)

In [137]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,0,NaN,2,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,0,NaN,0,2
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,0,NaN,2,0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,0,NaN,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,1,NaN,0,2


In [170]:
test.Age = test.Age.astype(np.int64)

In [208]:
X_test = df_test.iloc[:,:].values
X_test_var = Variable(torch.FloatTensor(X_test), requires_grad=False) 
with torch.no_grad():
    test_result = model(X_test_var)
values, labels = torch.max(test_result, 1)
survived = labels.data.numpy()

In [209]:
submission = pd.DataFrame({'PassengerId': df_sub['PassengerId'], 'Survived': survived})

In [210]:
submission.to_csv('torch_submission.csv', index=False)

In [ ]:
# Результат на kaggle 0.75598

In [211]:
torch.save(model.state_dict(),'torc.pth')

In [213]:
model = Net()
model.load_state_dict(torch.load('torc.pth'))

<All keys matched successfully>